# Attention is All You Need

## 1. Introduction 

자연어 처리 알고리즘 중 RNN, LSTM, GRU는 언어 모델링 및 기계 번역과 같은 문제에서 많은 성과를 이루었다. 하지만 Recurrent Model은 input sequences와 output sequences를 순차적으로 연산하기 때문에 병렬적으로 처리를 하지 못하고, sequence의 길이가 길어질수록 메모리 제약으로 인해 일괄적으로 처리하지 못하여 성능이 떨어진다.<br>
Attention mechanism은 이러한 문제를 해결하기 위한 방안으로 기존 RNN과 같이 사용하였지만 Transformers 논문에서는 RNN을 제거하고 Attention mechanism에 전적으로 의존하는 모델 아키텍쳐를 제안하였다.

## 2. Background

1. 순차 계산을 줄이기 위해 Extended Neural GPU, ByteNet과 ConvS2S를 기초로 형성하였는데 특징은 다음과 같다.
    
    1-1. Extended Neural GPU : sequential data에 대한 학습을  time-sequence로 학습하지 않고, 한 번에 받아 병렬적으로 처리하며 Encoder-Decoder의 구조를 가지고 있다. **sequence에 대해 key-value pair 형태로 저장하는 memory module을 도입하였으며, 주어진 query를 통해 유사한 순서대로 가중치**를 주어 학습을 진행한다.
    
    1-2. ByteNet : CNN을 사용하여 Encoder 위에 Decoder가 스택처럼 쌓이는 네트워크 구조를 만들며, dynamic unfolding을 통해 가변 길이 sequence를 생성한다. 이렇게 함으로써 기존 **Reccurent Model의 특징인 memorization이 필요 없게 된다. 또한 병렬화의 여지가 훨씬 크고 멀리 떨어진 단위 정보 사이의 관계특성을 더 잘 찾아 낼 수 있다.**
    
    1-3. ConvS2S : sequence to sequence 모델을 CNN으로 만든 모델이다. **Position Embeddings을 사용하여 input에 대하여 embedding vector와 절대적인 위치에 대한 embedding을 만든 후 Add하여 새로운 vector**를 만든다. encoder에서는 cnn층을 통과한 2d output을 element-wise multiply하고 **gated linear units**을 통과 한다. 이 때 **deep하게 만들기 위해 residual connection**을 사용함. decoder layer에 Attention mechanism을 적용시켰다.
    
    
2. 최초로 Self-Attention을 적용하여 RNN이나 CNN을 사용하지 않고 input과 output에 대한 representation을 compute 하였다.

## 3. Model Architecture

<img src='https://user-images.githubusercontent.com/74821253/112999648-17851e80-91aa-11eb-848b-5ec4ef7fabdc.JPG'>

Transformers의 전체적인 모델 구조는 다음과 같다. 

- 다른 sequences to sequences 모델들과 동일하게 전체적인 구조는 Encoder-Decoder구조를 가지고 있다.


- Encoder 내부에서 Attention mechanism, Decoder 내부에서 Attention mechanism, Encoder-Decoder 간의 Attention mechanism을 적용하였다.
<blockquote> 
    Attention mechanism : Decoder의 $i$번째 단어를 예측할 때 Encoder의 $j$번째 열백터와 직전 hidden state vector를 내적하여 유사도를 구한 후 가장 유사한 열벡터를 찾는 mechanism 이다.
    <figure>
        <img src='img/2.jpg' alt="Encoder" width="40%" height="40%" style="float:left;">
        <img src='img/3.jpg' alt="Decoder" width="30%" height="30%">
     
    </figure>
    
</blockquote>


- Query와 Key, value로 input을 구성하였고, Query와 key,value를 Scaled Dot-Product Attention을 진행해 주었고, 전체에 대하여 Attention mechanism을 적용시켜주는 대신에 Multi-Head Attention을 통해 h개의 Attention을 병렬적으로 적용하여 서로 다른 표현 공간들을 학습하고자 하였다.


- Encoder와 Decoder의 layer에는 Linear한 fully connected feed-forward network를 포함하고 있다.


- 학습가능한 Embeddings layer를 적용하였다. Embeddings layer는 input Embeddings layer output Embeddings layer가 존재하는데, 이 때 두 Embeddings layer는 같은 가중치를 공유한다.

- sequence에 대한 Embeddings layer 이외에도 Positional Encoding을 적용하여 위치 정보도 함께 전달해 주었다.

### 3-1. Encoder and Decoder Stacks

<img src='img/1.jpg'>

**Encoder :**  
- Encoder는 N개의 동일한 layer로 구성되어 있다.(논문에서는 6개의 layer를 쌓음)


- 각각의 layer는 2개의 sub-layers를 가지고 있다. 
    - Muti-head self-Attention mechanism 
    - Position-wise fully connected feed-forward network (여기서 position은 단어의 위치로 생각하면 됨)
    
    
- 위의 sub-layer 연산 후 residual connection을 이용하여 gradient vanishing 문제를 해결하였고, 이를 위해 sub-layer와 Embedding layer의 차원을 맞춰춘다.(논문에서는 512차원으로 맞춰주었다.) 그 후 Layer Normalization을 적용한다. 
  - $LayerNorm(x + Sublayer(x))$
  
 
**Decoder :**
- Decoder 또한 Encoder와 마찬가지로 N개의 동일한 layer로 구성되어 있다.(N = 6)


- Encoder의 layer와 동일하게 2개의 sub-layers를 가지고 있지만 Decoder는 추가적으로 Encoder의 output을 처리하기 위한 Multi-head Attention을 적용하였다.


- Encoder와 유사하게 각각의 sub-layer에 residual connection을 하였고 Layer Normalization을 적용하였다. 


- Decoder에서는 Encoder와 다르게 Masked self-attention을 적용하였다. 그 이유는 position i 보다 다음에 나오는 position에 attention을 주지 못하게 하기 위함이다.
<p>
<img src='img/4.jpg' style='float:left;' width="40%" height="40%">
<br>
옆의 예시를 보면, 0번째 위치인 a를 예측할 때는 1번째와 2번째의 b,c에 attention을 주지 않기 위해 masking 하였고 1번째 위치의 b를 예측할 때는 0번째 위치의 a에만 attention을 주기 위해 2번째 위치의 c에게는 주어지지 않게 masking을 하였다. 
    
**즉, Decoder는 Masking을 통해 순차적으로 단어를 예측하게 Attention을 적용한다.**
</p>


### 3-2. Attention

***Attention***

Attention은 input에 대하여 query와 key-value로 mapping을 한다. 출력에 대해서는 Query에 따라 유사한 key를 가지는 value의 가중합을 softmax를 통해 계산된다.(query, keys, values는 모두 vector 이다.)  

Transformers 논문에서는 Attention을 Scaled Dot-Product Attention으로 사용하였고, 방법은 아래와 같다.

<p>
<img src='img/5.jpg' width="40%" height="40%" style ="float:left;">
 
<img src='img/6.jpg' width="40%" height="40%">

</p>
<br>

***Scaled Dot-Product Attention :***  

- $d_{k}$의 queries와 keys $d_{v}$의 values로 이루어진 input에 대하여 $Q \cdot K^{T}$ 를 해준다. $Q \cdot K^{T}$ 내적을 통하여 어느 key가 유사도가 높은지 병렬적으로 수행히 가능하다.


- 이 때 $Q \cdot K^{T}$ 의 결과 값이 너무 커지지 않도록 $\sqrt{d_{k}}$로 나눠주어 scaling을 진행한다.
> why ? $Q \cdot K^{T}$의 Output logits에 대하여 softmax를 적용하여 가중치를 부여하는데, softmax함수를 생각해 보면 값이 커질수록 값이 극단적으로 한쪽으로 몰리고, gradient 값이 0에 가까워 지기 때문에 잘 이루어지지 않아 $\sqrt{d_{k}}$로 scaling을 진행함

- 위의 구조에서 Mask (opt.)라는 block이 있는데 이는 Attention 방향을 정해준다. Encoder에서는 input에 대하여 양방향으로 Attention을 적용시키지만, Decoder에서는 position i 보다 다음에 나오는 position에 attention을 주지 못하게 하기 위해 단방향으로 순차적으로 Attention을 적용시킨다.


- 마지막으로 앞에서 계산된 Scaled dot-product 값들을 softmax 함수에 통과 시켜 0~1 사이의 Attention weights를 구하여 values와의 점곱을 통해 values를 계산한다. 즉, Query와 유사한 Key의 Value일 수록 더 큰 값, 중요한 정보에 더욱 관심을 둔다.


- Attention에 주로 사용되는 연산은 Additive Attention과 Dot-Product Attention이 있는데 논문에서는 Dot-Product Attention을 적용하였다. Dot-Product Attention이 최적화된 Matrix MultiPly을 사용하여 더욱 빠르고 효율적이다.

<center>$Attention(Q,K,V) = softmax(\frac {Q \cdot K^{T}}{\sqrt{d_{k}}})V$</center>

- 자세한 방법은 아래와 같다. 
<p>
<img src='img/7.JPG' width = "80%" height="50%">
</p>
<br>



***Multi-Head Attention :***  

- 전체에 대하여 Attention mechanism을 적용시켜주지 않고 Query, Key-Value를 여러 가지로 변환하여 각각 다르게 학습된 linear projection을 h개의 Attention을 적용하였다. 각각의 Query, Key-Value에 대하여 Attention mechanism을 병렬적으로 적용시켰고, 마지막에 Concat 하여 Output을 산출하였다. (위의 Scaled Dot-Product Attention을 병렬적으로 h개 실행하여 결과를 concat 했다고 생각하면 됨.)

- Multi-Head Attention을 통해 다양한 위치에서 다양한 representation subspaces를 학습하였다. 즉, sequence에서 여러 가지 방식으로 여러 위치에 주의를 줄 수 있다.


<center>$MultiHead(Q,K,V) = Concat({head}_{1},{head}_{2} \cdots, {head}_{h})W^{O}$</center>
<center>where ${head}_{i} = Attention({{QW}_{i}}^{Q},{{KW}_{i}}^{K},{{VW}_{i}}^{V})$</center>

- 논문에서는 h = 8 로 병렬적으로 사용하였고, single-head attention의 computational cost와 유사하게 하기 위해 각 head의 $d_{k} = d_{v} = d_{model} / h = 64$ 에 맞게 차원을 축소하였다.




***Applications of Attention in out Model :***   

Transformers의 모델에서는 multi-head attention을 세 가지의 다른 방식으로 사용하였다.

- Encoder-Decoder : Encoder-Decoder 층에서는 Query를 이전 Decoder 층에서 가져오고 key-value는 Encoder의 output에서 가져왔다.


- Encoder : Encoder는 self-attention layer를 적용하여 query, key, value를 현재 input의 query, key, value에서 가져왔다. 즉 이전 층의 모든 position에 attention을 적용할 수 있다. 첫번째 Encoder의 Self-attention 이면 embedding layer + positional encoding에 대하여 attention을 적용한다.


- Decoder : Decoder 또한 Encoder와 마찬가지로 self-attention layer를 적용하였다. 다른 점은 i번째 output을 i+1번째 input으로 사용하는 auto-regressive 특성을 유지하기 위해 masking을 적용하여 i번째 이후에 있는 position에 attention을 주지 않게끔 하였다.

***Position-wise Feed-Forward Networks :***

- Encoder와 Decoder의 2번째 sub-layers는 fully connected feed-forward network로 이루어져 있는데, 이는 각 position마다 개별적으로 동일하게 적용된다. 이 sub-layer는 2개의 linear transformation과 ReLU로 이루어져 있다.

<center>$FFN(x) = max(0,xW_{1} + b_{1})W_{2} + b_{2}$</center>

- linear transformations의 가중치는 인코더 내에서는 동일한 값을 가지지만 인코더 층마다는 다른 값을 가진다. 즉, kernel size가 1인 2개의 CNN층으로 해석해도 된다. $W_{1}$은 ($d_{model}$, $d_{ff}$)의 크기를 가지고, $W_{2}$는 ($d_{ff}, d_{model}$)의 크기를 가진다. (논문에서는 $d_{model} = 512, d_{ff} = 2048$로 설정함.)

### 3-3. Embeddings

**Embeddings layer and Positional Encoding : ***

- 다른 sequence to sequence 모델과 유사하게 학습이 가능한 Embedding layer를 사용하여 $d_{model}$의 차원으로 변환한다. input과 output의 embeddings layer는 가중치가 동일하고 공유가 가능하다. 또한 decoder의 output을 다음 단어의 확률로 바꾸기 위해 learned linear transformation과 softmax를 사용하였다.


- Transformers는 Embeddings 뿐만 아니라 Positional Encoding을 진행하였는데, transformers는 reccurence와 convolution을 사용하지 않기 때문에 sequence의 token에 대한 절대적이나 상대적인 위치를 반드시 주어야 한다. 논문에서는 다양한 Positional Encoding을 시도해 보았지만 **sinusoid**를 이용한 Positional Encoding을 사용하였다.

<center>$PE(pos, 2i) = sin(pos/{10000}^{2i/d_{model}})$</center>
    <center>$PE(pos, 2i+1) = cos(pos/{10000}^{2i/d_{model}})$    $i$ : dimension, $pos$ : position</center> 
    
    
> 논문 저자는 Positional Encoding에 learned positional embeddings 도 실험해 보았지만, sinusoid를 이용했을 때와 동일한 결과가 나왔다고 하였다. 하지만 sinusoid를 이용한 Positional Encoding이 학습을 할 때 등장한 sequence 길이보다 더 긴 sequence에 대해서도 처리를 할 수 있기 때문이다. (가변적인 sequence에 특성에 더 잘 맞는 방법이 sinusoid를 이용한 Positional Encoding이라는 의미이다.)

- 위의 식은 논문에서 제시한 Positional Encoding 방법인데 dimension이 짝수일 경우 sin함수를 홀수일 경우는 cos함수를 사용한다. sin,cos함께 사용하는 이유는 cos함수는 sin함수만 사용했을 경우 같은 Postional Encoding 값이 나올 수 있기 때문이다. 또 논문에서는 고정된 k만큼 거리에 대해 $PE_{pos+k}$를 $PE_{pos}$의 선형 함수로 표현될 수 있기 때문에 상대적인 위치를 쉽게 구할 수 있다고 생각하였다.
> 가변적인 sequence에 대하여 i+k번째 PE값을 i번째 PE값으로 구할 수 있다고 생각한거 같다. 


- 아래는 위의 두 식을 합쳐 시각화를 나타낸 결과이다. Depth만큼의 sin과 cos함수를 만들어 내어 각 Position에 대한 Encoding을 Embeddings layer의 값과 Add 하여 input을 만들어 낸다. 


<img src='img/8.JPG' width = "80%" height="50%">

## 4. Why Self-Attention

- 각 층의 layer 계산 복잡도 : input의 sequence 길이 n이 Embedding 차원 d보다 작은 경우가 대부분이다. 아래의 계산 복잡도를 비교해보면, Self-Attention이 Recurrent나 Convolutional에 비해 계산 복잡도가 낮은것을 확인할 수 있다.


- minimum number of sequential operation에 대한 병렬적 연산이 가능한 계산량 : sequential한 operation이 $O(1)$의 복잡도를 가진다.


- network의 long-range dependencies 사이의 경로 길이 : Self-Attention은 Reccurent나 Convolutional과 다르게 sequence 내의 모든 단어에 한번에 접근하기 때문에 $O(1)$의 복잡도를 가진다.

<img src='img/9.JPG' width = "80%" height="50%">

- 추가적으로, Attention distributions를 통해 학습이 진행되기 때문에 각각의 attention head는 문장내의 syntactic하고 semantic한 구조와 관련된 특징을 포착하여 모델의 설명력을 높여준다.

## 5. Training

- WMT 2014 English-German Dataset(4.5 million sentence pairs)으로 학습을 진행하였고 BPE 알고리즘을 통해 encoding을 하였다. 25000개의 source tokens와 25000개의 target tokens를 batch로 사용하였다. 


- 8 NVIDIA P100 GPUs 사용하여 12시간동안 100,000 steps를 학습하였고, Big models는 300,000 steps를 학습하였다.


- Adam optimizer($\beta_{1} = 0.9, \beta_{2} = 0.98 , \epsilon = 10^{-9}$)를 사용하였고, learning rate는 scheduler를 사용하여 아래 식과 같이 적용하였다.
<center>$lrate = d_{model}^{-0.5} \cdot min({step\_num}^{-0.5}, step\_num \cdot {warmup\_steps}^{-1.5})$</center>


- Residual Dropout($P_{drop} = 0.1$) 과 Label Smoothing($\epsilon_{ls} = 0.1$)을 사용하였다

## 6. Results

- WMT English-to-German 데이터에 대해서 이전 SOTA 보다 2.0 큰 28.4의 BLEU를 달성하였다. 

- WMT English-to-French 데이터에 대해서는 big model이 BLEU 41.0 을 달성하였다.

<img src='img/10.JPG' width = "80%" height="50%">

## 7. Conclusion

이 연구를 통해, Encoder-Decoder 구조에서 일반적으로 사용되는 RNN을 Multi-Head Attention으로 대체 하였고, 전적으로 Attention을 기반으로한 첫 sequence to sequence 모델을 제시하였다. 

Translation tasks에 대해 Reccurrent 나 Convolutional 모델 보다 훨씬 빠르게 학습을 할 수 있고 WMT English-to-German, English-to-French에서 SOTA를 달성할 수 있었다.

또한, 텍스트 뿐만 아니라 이미지, 오디오 및 비디오 같은 큰 input과 output에 대해 효율적인 restricted attention mechanisms를 연구할 계획이다.

#### 참고자료

Extended Neural GPU : https://blog.lunit.io/2017/03/27/learning-to-remember-rare-events/

bytenet : https://brunch.co.kr/@kakao-it/155

convs2s : https://reniew.github.io/44/

Attention : https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/10/06/attention/

transformers : https://pozalabs.github.io/transformer/ 
                https://wikidocs.net/31379